In [4]:
import os

# Ruta a la carpeta con tus CSVs
carpeta = r'C:\Users\Elena\Desktop\GitHub\TFG-Elena-Ruiz\Datos\Datos crudos\save_log\señales no válidadas'

for archivo in os.listdir(carpeta):
    if archivo.endswith(".csv"):
        ruta = os.path.join(carpeta, archivo)
        with open(ruta, 'r', encoding='utf-8') as f:
            contenido = f.read().replace(';', ',')
        with open(ruta, 'w', encoding='utf-8') as f:
            f.write(contenido)

print("¡Todos los archivos han sido actualizados con comas como separador!")


¡Todos los archivos han sido actualizados con comas como separador!
